In [112]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from constants import SUBJECTS, ELECTRODES, FEATURE_FILENAME, RESULTS_FOLDER
from configs import configs

In [113]:
features = pd.read_csv(RESULTS_FOLDER / FEATURE_FILENAME)
print(features.shape)

threshold = 4 #in std
features.loc[:,ELECTRODES] = features.loc[:,ELECTRODES][features.loc[:,ELECTRODES]<threshold]
features = features.dropna()
print(features.shape)
features = features[features['game'] != '-']
#features = features[features['band'] != 'beta']

(53202, 14)
(52330, 14)


In [129]:
import math
import scipy

electrodes = ['T7', 'O1', 'C1', 'Fp1', 'O2', 'C2', 'Fp2', 'T8']

features['fron'] = (features['Fp1'] + features['Fp2'])/2
features['occ'] = (features['O1'] + features['O2'])/2
features['par'] = (features['C1'] + features['C2'])/2

features['all'] = features[electrodes].mean(axis=1)


def determine_signif(p,thresh):
    if p < thresh*1e-2:
        return "***"
    if p < thresh*1e-1:
        return "**"
    if p < thresh:
        return "*"
    else:
        return '-'

def minigame_stats(games, feature, thresh=0.05, d='greater'):
    
    version1 = games[games["difficulty"] == "1"]
    version2 = games[games["difficulty"] == "2"]
    version3 = games[games["difficulty"] == "3"]

    v1_v2_challenge_pvalue = scipy.stats.ttest_ind(version2[feature], version1[feature],
                                                  alternative=d)[1]
    v1_v3_challenge_pvalue = scipy.stats.ttest_ind(version3[feature], version1[feature],
                                                  alternative=d)[1]
    
    sign = '<' if d == 'less' else '>'

    return pd.Series({"baseline":version1[feature].mean(),
                      "mechanical":version3[feature].mean(),
                      "visual":version2[feature].mean(),
                      "test-mech": determine_signif(v1_v3_challenge_pvalue,thresh),
                      "test-vis": determine_signif(v1_v2_challenge_pvalue,thresh),
                     })

num_tests = 4
thresh = 1-math.pow(1-0.05,1/num_tests)

style_dic = {'baseline': "{:.3f}",
             'visual': "{:.3f}",
             'mechanical': "{:.3f}"
            }

for dire in ['greater', 'less']:
    if dire == 'less':
        band = 'alpha'
    else:
        band = 'gamma'

    test = features.groupby(['game','band']).apply(lambda x: minigame_stats(x,'fron',thresh=thresh,d=dire))
    df = test.reset_index()
    df = df[df['band'] == band]
    df = df.drop(columns='band')
    df.reset_index(drop=True)
    print(df.style.hide(axis="index").format(style_dic).to_latex(caption=f'{band} band power',
                                                                 label=f'{band}_data',
                                                                 position='h'))

\begin{table}[h]
\caption{gamma band power}
\label{gamma_data}
\begin{tabular}{lrrrll}
game & baseline & mechanical & visual & test-mech & test-vis \\
DOBBLE & -0.042 & 0.045 & 0.003 & *** & * \\
FLAPPY & -0.284 & -0.184 & -0.274 & *** & - \\
RUNSHOOT & -0.091 & -0.018 & -0.038 & *** & ** \\
SORTER & 0.068 & 0.128 & -0.003 & *** & - \\
\end{tabular}
\end{table}

\begin{table}[h]
\caption{alpha band power}
\label{alpha_data}
\begin{tabular}{lrrrll}
game & baseline & mechanical & visual & test-mech & test-vis \\
DOBBLE & 0.104 & 0.080 & 0.046 & - & * \\
FLAPPY & -0.213 & -0.173 & -0.218 & - & - \\
RUNSHOOT & -0.156 & -0.140 & -0.134 & - & - \\
SORTER & -0.009 & -0.033 & 0.045 & - & - \\
\end{tabular}
\end{table}



In [121]:
gameplay_quest = pd.read_csv("../Raw/subject_data.csv")
gameplay_quest["Challenge"] = gameplay_quest[["Q1", "Q2", "Q3", "Q4"]].sum(axis=1)

In [127]:
def determine_signif(p,thresh):
    if p < thresh*1e-2:
        return "***"
    if p < thresh*1e-1:
        return "**"
    if p < thresh:
        return "*"
    else:
        return '-'

def minigame_stats(games, feature="Challenge", thresh=0.05):
    
    version1 = games[games["Version"] == 1]
    version2 = games[games["Version"] == 2]
    version3 = games[games["Version"] == 3]

    v1_v2_challenge_pvalue = scipy.stats.ttest_ind(version2[feature], version1[feature])[1]
    v1_v3_challenge_pvalue = scipy.stats.ttest_ind(version3[feature], version1[feature])[1]

    return pd.Series({"baseline":version1[feature].mean(),
                      "mechanical":version3[feature].mean(),
                      "visual":version2[feature].mean(),
                      "test-mech": determine_signif(v1_v3_challenge_pvalue,thresh),
                      "test-vis": determine_signif(v1_v2_challenge_pvalue,thresh),
                     })

num_tests = 2
thresh = 1-math.pow(1-0.05,1/num_tests)

style_dic = {'baseline': "{:.3f}",
             'visual': "{:.3f}",
             'mechanical': "{:.3f}"
            }


test = gameplay_quest.groupby(by=['MiniGame']).apply(lambda x: minigame_stats(x,thresh=thresh))
df = test.reset_index()
df = df.rename(columns={"MiniGame": "game"})
df.reset_index(drop=True)
print(df.style.hide(axis="index").format(style_dic).to_latex(caption='Perceived difficulty score',
                                                             label='questionnaire_data',position='h'))

\begin{table}[h]
\caption{Perceived difficulty score}
\label{questionnaire_data}
\begin{tabular}{lrrrll}
game & baseline & mechanical & visual & test-mech & test-vis \\
DOBBLE & 14.292 & 14.958 & 13.667 & - & - \\
FLAPPY & 10.083 & 10.875 & 10.417 & - & - \\
RUNSHOOT & 12.417 & 14.667 & 11.750 & * & - \\
SORTER & 13.333 & 15.042 & 12.542 & - & - \\
\end{tabular}
\end{table}

